### Description

These are not comprehensive tests. We also need to vary the environment for 'target_expression'

---

In [2]:
import pandas as pd

from GA.ReinforcementLearning import *
from function_modules.function_module import *

In [3]:
cnn_model_path = '../../Models/CNN_6_1_2.keras'
masked_sequence = 'AATACTAGAGGTCTTCCGACNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNGTGTGGGCGGGAAGACAACTAGGGG'
target_expression = 1

In [34]:
# Generate the masked sequences based on pLac elements
pLac_UP = 'AATACTAGAGGTCTTCCGAC'
pLac_h35 = 'TTTACA'
pLac_spacer = 'CTTTATGCTTCCGGCTCG'
pLac_h10 = 'TATGTT'
pLac_disc = 'GTGTGGGC'
pLac_ITR = 'GGGAAGACAACTAGGGG'

# All pLac elements in order
pLac_elements = [pLac_UP, pLac_h35, pLac_spacer, pLac_h10, pLac_disc, pLac_ITR]

# Mask each pLac element
masked_sequences = []
for element in pLac_elements:
    i = pLac_elements.index(element)
    up = ''.join([pLac_elements[j] for j in range(i)])
    mask = 'N' * len(pLac_elements[i])
    down = ''.join([pLac_elements[j] for j in range(i+1, len(pLac_elements))])
    masked_sequences.append(up + mask + down)

# Mask (h35 + h10) and (h35 + spacer + h10)
masked_sequences.append(pLac_UP + 'N'*len(pLac_h35) + pLac_spacer + 'N'*len(pLac_h10) + pLac_disc + pLac_ITR)
masked_sequences.append(pLac_UP + 'N'*len(pLac_h35) + 'N'*len(pLac_spacer) + 'N'*len(pLac_h10) + pLac_disc + pLac_ITR)

masked_sequences

['NNNNNNNNNNNNNNNNNNNNTTTACACTTTATGCTTCCGGCTCGTATGTTGTGTGGGCGGGAAGACAACTAGGGG',
 'AATACTAGAGGTCTTCCGACNNNNNNCTTTATGCTTCCGGCTCGTATGTTGTGTGGGCGGGAAGACAACTAGGGG',
 'AATACTAGAGGTCTTCCGACTTTACANNNNNNNNNNNNNNNNNNTATGTTGTGTGGGCGGGAAGACAACTAGGGG',
 'AATACTAGAGGTCTTCCGACTTTACACTTTATGCTTCCGGCTCGNNNNNNGTGTGGGCGGGAAGACAACTAGGGG',
 'AATACTAGAGGTCTTCCGACTTTACACTTTATGCTTCCGGCTCGTATGTTNNNNNNNNGGGAAGACAACTAGGGG',
 'AATACTAGAGGTCTTCCGACTTTACACTTTATGCTTCCGGCTCGTATGTTGTGTGGGCNNNNNNNNNNNNNNNNN',
 'AATACTAGAGGTCTTCCGACNNNNNNCTTTATGCTTCCGGCTCGNNNNNNGTGTGGGCGGGAAGACAACTAGGGG',
 'AATACTAGAGGTCTTCCGACNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNGTGTGGGCGGGAAGACAACTAGGGG']

In [5]:
# Train the RL agent with multiple sequences
model = train_rl_agent(cnn_model_path, masked_sequences, target_expression, total_timesteps=1000, num_envs=4)
model.save('../../Models/RL_1_0')

Using cpu device
-----------------------------
| time/              |      |
|    fps             | 163  |
|    iterations      | 1    |
|    time_elapsed    | 50   |
|    total_timesteps | 8192 |
-----------------------------


In [6]:
# Test the RL agent on the (h35 + spacer + h10) masked sequence
test_sequence = masked_sequences[-1]
print(test_sequence)

AATACTAGAGGTCTTCCGACNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNGTGTGGGCGGGAAGACAACTAGGGG


In [35]:
# Load the pre-trained model
model = load_rl_agent('../../Models/RL_1_0')

target_expressions = np.linspace(0, 1, 11)

# Optimize a new sequence using the trained model for each target expression
for target_expression in target_expressions:
    sequence, prediction, error, encoded = optimize_sequence_with_rl(model, cnn_model_path, masked_sequence, target_expression)
    # print(sequence)
    print(prediction)
    # print(error)

0.2826947
0.30730295
0.326266
0.33123773
0.314807
0.3856442
0.32230258
0.29798675
0.3576943
0.33402652
0.27310663
